In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
# from langchain_openai import ChatOpenAI
# from langchain_ollama import ChatOllama
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType, Tool
import re
from datetime import time
from langchain.schema import HumanMessage
import openai
import requests
import time
import numpy as np
import os
import pandas as pd
import sys
import time
from io import StringIO
import contextlib
import gurobipy as gp
from gurobipy import GRB
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from typing import List
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from typing import List, Dict
from langchain_openai import ChatOpenAI
user_api_key = "YOUR_OPENAI_API_KEY"  # Replace with your OpenAI API key

### Functions

In [ ]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from typing import List

# def build_llm(model: str = "gpt-oss:20b", temperature: float = 0.0) -> ChatOllama:
#     """Initializes the ChatOllama model instance."""
#     return ChatOllama(
#         model=model,
#         temperature=temperature,
#         base_url="http://localhost:11434",
#         timeout=500
#     )


# def build_llm(model: str = "gpt-4.1", temperature: float = 0.0) -> ChatOpenAI:
#     return ChatOpenAI(
#         model=model,
#         temperature=temperature,
#         openai_api_key=user_api_key
#     )

# def build_llm(model: str = "gpt-5", temperature: float = 1.0) -> ChatOpenAI:
#     return ChatOpenAI(
#         model=model,
#         temperature=temperature,
#         openai_api_key=user_api_key
#     )

def build_llm(model: str = "gpt-5.2", temperature: float = 0.0) -> ChatOpenAI:
    return ChatOpenAI(
        model=model,
        temperature=temperature,
        openai_api_key=user_api_key
    )

def run_test(df: pd.DataFrame, llm: ChatOpenAI) -> List[str]:
    """
    Runs the test by sending queries from the DataFrame to the LLM.
    
    Args:
        df: DataFrame containing a 'Query' column.
        llm: The initialized ChatOllama instance.

    Returns:
        A list of string responses from the LLM.
    """
    result = []

    for row_idx, row in df.iterrows():
        query = row['Query'] 
        
        prompt = f"""
Below is an operations research question. Build a mathematical model and corresponding python code using 'gurobipy' that appropriately addresses the question.

# Question
{query}

# Response
        """

        messages = [
            HumanMessage(content=prompt)
        ]

        try:
            response = llm(messages)
            response_content = response.content
            
            print(response_content)
            
            result.append(response_content)
            
        except Exception as e:
            print(f"Error processing query at Index {row_idx}: {e}")
            result.append(f"Error: {e}") 
            
    return result

llm1 = build_llm()


In [ ]:
def read_and_combine_csvs(file_order):
    dfs = []
    for fname in file_order:
        if os.path.exists(fname):
            df = pd.read_csv(fname)
            dfs.append(df)
            print(f"Read file: {fname} (Row length: {len(df)})")
        else:
            print(f"File doesn't exist: {fname}, already skipped")
    
    if not dfs:
        raise ValueError("No effective files")
    
    return pd.concat(dfs, ignore_index=True)

def run_gurobi_code(code_str):
 
    try:
      
        with StringIO() as buf, contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
            env = {
                '__builtins__': __builtins__,
                'gp': gp,
                'GRB': GRB
            }
            
           
            code_str += "\n\n# Added by executor\n"
            code_str += "if hasattr(m, 'status') and m.status == GRB.OPTIMAL:\n"
            code_str += "    __result__ = m.ObjVal\n"
            code_str += "else:\n"
            code_str += "    __result__ = None\n"
            
            
            exec(code_str, env)
            result = env.get('__result__', None)
            
     
            if 'm' in env:
                env['m'].dispose()
                del env['m']
            
            return result
    except Exception as e:
        print(f"Execution error: {str(e)}")
        return None


In [ ]:
def extract_python_code(text):
    pattern = r'```python(.*?)```'
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def run_gurobi_code(code):
    try:
        
        local_vars = {'gp': gp, 'GRB': GRB, '__builtins__': __builtins__}
        
        exec(code, local_vars)
        
        models = []
        for var_name, var_value in local_vars.items():
            if isinstance(var_value, gp.Model):
                models.append(var_value)
        
        if models:
            model = models[-1] 
            if model.status == GRB.OPTIMAL:
                return model.objVal
            else:
                print(f"Model status is not optimal: {model.status}")
                return None
        
        return None
    except Exception as e:
        print(f"Error running Gurobi code: {e}")
        return None

### Test NL4OPT

In [ ]:
test_nl4opt = pd.read_csv('Test_Dataset/Small-scale/NL4OPT NEW.csv')
test_nl4opt1=test_nl4opt[:50]
test_nl4opt2=test_nl4opt[50:100]
test_nl4opt3=test_nl4opt[100:150]
test_nl4opt4=test_nl4opt[150:200]
test_nl4opt5=test_nl4opt[200:]
test_nl4opt


In [ ]:
output_model_nl4opt1 = run_test(test_nl4opt1,llm1)
output_df = pd.DataFrame({'Query': test_nl4opt1['Query'], 'model_output':output_model_nl4opt1})
output_df.to_csv("nl4opt_1-50_5.2.csv", index=False)

In [ ]:
output_model_nl4opt2 = run_test(test_nl4opt2,llm1)
output_df = pd.DataFrame({'Query': test_nl4opt2['Query'], 'model_output':output_model_nl4opt2})
output_df.to_csv("nl4opt_51-100_5.2.csv", index=False)

In [ ]:
output_model_nl4opt3 = run_test(test_nl4opt3,llm1)
output_df = pd.DataFrame({'Query': test_nl4opt3['Query'], 'model_output':output_model_nl4opt3})
output_df.to_csv("nl4opt_101-150_5.2.csv", index=False)

In [ ]:
output_model_nl4opt4 = run_test(test_nl4opt4,llm1)
output_df = pd.DataFrame({'Query': test_nl4opt4['Query'], 'model_output':output_model_nl4opt4})
output_df.to_csv("nl4opt_151-200_5.2.csv", index=False)

In [ ]:
output_model_nl4opt5= run_test(test_nl4opt5,llm1)
output_df = pd.DataFrame({'Query': test_nl4opt5['Query'], 'model_output':output_model_nl4opt5})
output_df.to_csv("nl4opt_201-_5.2.csv", index=False)

In [ ]:
file_order=[
    "nl4opt_1-50_5.2.csv",
    "nl4opt_51-100_5.2.csv",
    "nl4opt_101-150_5.2.csv",
    "nl4opt_151-200_5.2.csv",
    "nl4opt_201-_5.2.csv",
]
try:
    combined_df = read_and_combine_csvs(file_order)
    combined_df.to_csv('nl4opt_5.2_Base_Model.csv')
    print(f"Total rows: {len(combined_df)}")
except Exception as e:
    print(f"File processing failed: {str(e)}")
    sys.exit(1)
    
print("\nRunning Gurobi Code...")
objective_values = []
start_time = time.time()
success_count = 0

for i, row in combined_df.iterrows():
    full_text = row['model_output']
    
    code = extract_python_code(full_text)
    
    if code:
        print(f"Processing row {i+1}/{len(combined_df)}...", end='\r')
        result = run_gurobi_code(code)
        objective_values.append(result)
        if result is not None:
            success_count += 1
    else:
        print(f"Warning: No Python code found in row {i+1}")
        objective_values.append(None)

combined_df['best_objective'] = objective_values

print(f"\nCode execution completed! Time used: {time.time()-start_time:.2f} seconds")
print(f"Success: {success_count}/{len(combined_df)}")

print("\nFirst few objective values:")
for i in range(min(5, len(combined_df))):
    print(f"Row {i+1}: Best objective = {objective_values[i]}")

output_file = "nl4opt_5.2_Base_Model.xlsx"
combined_df.to_excel(output_file, index=False)
print(f"\nResults saved to {output_file}")


### Test IndustryOR

In [ ]:
test_industryOR = pd.read_csv('Test_Dataset/Small-scale/IndustryOR_fixedV2.csv')
test_industryOR1_=test_industryOR[:25]
test_industryOR2=test_industryOR[25:50]
test_industryOR3=test_industryOR[50:75]
test_industryOR4=test_industryOR[75:]
test_industryOR

In [ ]:
output_model_industryOR1= run_test(test_industryOR1_,llm1)
output_df = pd.DataFrame({'Query': test_industryOR1_['Query'], 'model_output':output_model_industryOR1})
output_df.to_csv("IndustryOR_1-25_5.2.csv", index=False)

In [ ]:
output_model_industryOR2 = run_test(test_industryOR2,llm1)
output_df = pd.DataFrame({'Query': test_industryOR2['Query'], 'model_output':output_model_industryOR2})
output_df.to_csv("IndustryOR_26-50_5.2.csv", index=False)

In [ ]:
output_model_industryOR3 = run_test(test_industryOR3,llm1)
output_df = pd.DataFrame({'Query': test_industryOR3['Query'], 'model_output':output_model_industryOR3})
output_df.to_csv("IndustryOR_51-75_5.2.csv", index=False)

In [ ]:
output_model_industryOR4 = run_test(test_industryOR4,llm1)
output_df = pd.DataFrame({'Query': test_industryOR4['Query'], 'model_output':output_model_industryOR4})
output_df.to_csv("IndustryOR_76-100_5.2.csv", index=False)

In [ ]:
file_order=[
    "IndustryOR_1-25_5.2.csv",
    "IndustryOR_26-50_5.2.csv",
    "IndustryOR_51-75_5.2.csv",
    "IndustryOR_76-100_5.2.csv",
]
try:
    combined_df = read_and_combine_csvs(file_order)
    print(f"Total rows: {len(combined_df)}")
    output_file = "IndustryOR_result_5.2.csv"
    combined_df.to_csv(output_file, index=False)
except Exception as e:
    print(f"File processing failed: {str(e)}")
    sys.exit(1)


print("\nRunning Gurobi Code...")
objective_values = []
start_time = time.time()
success_count = 0

for i, row in combined_df.iterrows():
    full_text = row['model_output']
    
    code = extract_python_code(full_text)
    
    if code:
        print(f"Processing row {i+1}/{len(combined_df)}...", end='\r')
        result = run_gurobi_code(code)
        objective_values.append(result)
        if result is not None:
            success_count += 1
    else:
        print(f"Warning: No Python code found in row {i+1}")
        objective_values.append(None)

combined_df['best_objective'] = objective_values

print(f"\nCode execution completed! Time used: {time.time()-start_time:.2f} seconds")
print(f"Success: {success_count}/{len(combined_df)}")

print("\nFirst few objective values:")
for i in range(min(5, len(combined_df))):
    print(f"Row {i+1}: Best objective = {objective_values[i]}")

output_file = "IndustryOR_result_5.2.xlsx"
combined_df.to_excel(output_file, index=False)
print(f"\nResults saved to {output_file}")

### Test MAMO easy

In [ ]:
test_MAMOe = pd.read_csv('Test_Dataset/Small-scale/MAMO_EasyLP_fixed.csv')
test_MAMOe1=test_MAMOe[:100]
test_MAMOe2=test_MAMOe[100:200]
test_MAMOe3=test_MAMOe[200:300]
test_MAMOe4=test_MAMOe[300:400]
test_MAMOe5=test_MAMOe[400:500]
test_MAMOe6=test_MAMOe[500:600]
test_MAMOe7=test_MAMOe[600:]

In [ ]:
output_model_MAMOe1 = run_test(test_MAMOe1,llm1)
output_df = pd.DataFrame({'Query': test_MAMOe1['Query'], 'model_output':output_model_MAMOe1})
output_df.to_csv("MAMOe1_5.2.csv", index=False)

In [ ]:
output_model_MAMOe2 = run_test(test_MAMOe2,llm1)
output_df = pd.DataFrame({'Query': test_MAMOe2['Query'], 'model_output':output_model_MAMOe2})
output_df.to_csv("MAMOe2_5.2.csv", index=False)

In [ ]:
output_model_MAMOe3 = run_test(test_MAMOe3,llm1)
output_df = pd.DataFrame({'Query': test_MAMOe3['Query'], 'model_output':output_model_MAMOe3})
output_df.to_csv("MAMOe3_5.2.csv", index=False)

In [ ]:
output_model_MAMOe4 = run_test(test_MAMOe4,llm1)
output_df = pd.DataFrame({'Query': test_MAMOe4['Query'], 'model_output':output_model_MAMOe4})
output_df.to_csv("MAMOe4_5.2.csv", index=False)

In [ ]:
output_model_MAMOe5 = run_test(test_MAMOe5,llm1)
output_df = pd.DataFrame({'Query': test_MAMOe5['Query'], 'model_output':output_model_MAMOe5})
output_df.to_csv("MAMOe5_5.2.csv", index=False)

In [ ]:
output_model_MAMOe6 = run_test(test_MAMOe6,llm1)
output_df = pd.DataFrame({'Query': test_MAMOe6['Query'], 'model_output':output_model_MAMOe6})
output_df.to_csv("MAMOe6_5.2.csv", index=False)

In [ ]:
output_model_MAMOe7 = run_test(test_MAMOe7,llm1)
output_df = pd.DataFrame({'Query': test_MAMOe7['Query'], 'model_output':output_model_MAMOe7})
output_df.to_csv("MAMOe7_5.2.csv", index=False)

In [ ]:
file_order=[
    "MAMOe1_5.2.csv",
    "MAMOe2_5.2.csv",
    "MAMOe3_5.2.csv",
    "MAMOe4_5.2.csv",
    "MAMOe5_5.2.csv",
    "MAMOe6_5.2.csv",
    "MAMOe7_5.2.csv",
]
try:
    combined_df = read_and_combine_csvs(file_order)
    print(f"Total rows: {len(combined_df)}")
except Exception as e:
    print(f"File processing failed: {str(e)}")
    sys.exit(1)
    
print("\nRunning Gurobi Code...")
objective_values = []
start_time = time.time()
success_count = 0

for i, row in combined_df.iterrows():
    full_text = row['model_output']
    
    code = extract_python_code(full_text)
    
    if code:
        print(f"Processing row {i+1}/{len(combined_df)}...", end='\r')
        result = run_gurobi_code(code)
        objective_values.append(result)
        if result is not None:
            success_count += 1
    else:
        print(f"Warning: No Python code found in row {i+1}")
        objective_values.append(None)

combined_df['best_objective'] = objective_values

print(f"\nCode execution completed! Time used: {time.time()-start_time:.2f} seconds")
print(f"Success: {success_count}/{len(combined_df)}")

print("\nFirst few objective values:")
for i in range(min(5, len(combined_df))):
    print(f"Row {i+1}: Best objective = {objective_values[i]}")

output_file = "MAMOe_Base_5.2.xlsx"
combined_df.to_excel(output_file, index=False)
print(f"\nResults saved to {output_file}")

### Test MAMO Complex

In [ ]:
test_MAMOc = pd.read_csv('Test_Dataset/Small-scale/MAMO_ComplexLP_fixed.csv')
test_MAMOc1=test_MAMOc[:25]
test_MAMOc2=test_MAMOc[25:50]
test_MAMOc3=test_MAMOc[50:75]
test_MAMOc4=test_MAMOc[75:100]
test_MAMOc5=test_MAMOc[100:125]
test_MAMOc6=test_MAMOc[125:150]
test_MAMOc7=test_MAMOc[150:175]
test_MAMOc8=test_MAMOc[175:]
test_MAMOc8

classify_problem = llm1

In [ ]:
output_model_MAMOc1 = run_test(test_MAMOc1,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc1['Query'], 'model_output':output_model_MAMOc1})
output_df.to_csv("MAMOc_1-25_5.2.csv", index=False)

In [ ]:
output_model_MAMOc2 = run_test(test_MAMOc2,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc2['Query'], 'model_output':output_model_MAMOc2})
output_df.to_csv("MAMOc_26-50_5.2.csv", index=False)

In [ ]:
output_model_MAMOc3 = run_test(test_MAMOc3,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc3['Query'], 'model_output':output_model_MAMOc3})
output_df.to_csv("MAMOc_51-75_5.2.csv", index=False)

In [ ]:
output_model_MAMOc4 = run_test(test_MAMOc4,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc4['Query'], 'model_output':output_model_MAMOc4})
output_df.to_csv("MAMOc_76-100_5.2.csv", index=False)

In [ ]:
output_model_MAMOc6 = run_test(test_MAMOc5,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc5['Query'], 'model_output':output_model_MAMOc6})
output_df.to_csv("MAMOc_101-125_5.2.csv", index=False)

In [ ]:
output_model_MAMOc6 = run_test(test_MAMOc6,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc6['Query'], 'model_output':output_model_MAMOc6})
output_df.to_csv("MAMOc_126-150_5.2.csv", index=False)

In [ ]:
output_model_MAMOc7 = run_test(test_MAMOc7,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc7['Query'], 'model_output':output_model_MAMOc7})
output_df.to_csv("MAMOc_151-175_5.2.csv", index=False)

In [ ]:
output_model_MAMOc8 = run_test(test_MAMOc8,classify_problem)
output_df = pd.DataFrame({'Query': test_MAMOc8['Query'], 'model_output':output_model_MAMOc8})
output_df.to_csv("MAMOc_176-202_5.2.csv", index=False)

In [ ]:
file_order=[
    "MAMOc_1-25_5.2.csv",
    "MAMOc_26-50_5.2.csv",
    "MAMOc_51-75_5.2.csv",
    "MAMOc_76-100_5.2.csv",
    "MAMOc_101-125_5.2.csv",
    "MAMOc_126-150_5.2.csv",
    "MAMOc_151-175_5.2.csv",
    "MAMOc_176-202_5.2.csv",
]
try:
    combined_df = read_and_combine_csvs(file_order)
    print(f"Total rows: {len(combined_df)}")
    output_file = "MAMOc_result_5.2.csv"
    combined_df.to_csv(output_file, index=False)
except Exception as e:
    print(f"File processing failed: {str(e)}")
    sys.exit(1)
    
print("\nRunning Gurobi Code...")
objective_values = []
start_time = time.time()
success_count = 0

for i, row in combined_df.iterrows():
    full_text = row['model_output']
    
    code = extract_python_code(full_text)
    
    if code:
        print(f"Processing row {i+1}/{len(combined_df)}...", end='\r')
        result = run_gurobi_code(code)
        objective_values.append(result)
        if result is not None:
            success_count += 1
    else:
        print(f"Warning: No Python code found in row {i+1}")
        objective_values.append(None)

combined_df['best_objective'] = objective_values

print(f"\nCode execution completed! Time used: {time.time()-start_time:.2f} seconds")
print(f"Success: {success_count}/{len(combined_df)}")

print("\nFirst few objective values:")
for i in range(min(5, len(combined_df))):
    print(f"Row {i+1}: Best objective = {objective_values[i]}")

output_file = "MAMOc_result_5.2.xlsx"
combined_df.to_excel(output_file, index=False)
print(f"\nResults saved to {output_file}")